# **Import Library**

In [1]:
import pandas as pd
from pandas import ExcelFile
import numpy as np
import tensorflow as tf

# **Connect to Drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Data Preprocessing**
Use Harris-Benedic Theorem to process user calorie needed

In [32]:
food_data=pd.read_csv('/content/drive/MyDrive/bangkit3/Dataset Makanan2.csv')
snack_data= pd.read_csv('/content/drive/MyDrive/bangkit3/DatasetSnack.csv')
dataset_pengguna=pd.read_csv('/content/drive/MyDrive/bangkit3/dataset_pengguna4.csv')
# Convert the DataFrame to a dictionary
food_database = food_data.to_dict(orient='index')
snack_database = snack_data.to_dict(orient='index')

In [33]:
food_data

,No,Food Name,Calorie,Carbohidrat,Fat,Protein,Price
0,1,Mie aceh rebus,113,3,3,18,"15,000"
1,2,Mie ayam,102,6,4,11,"10,000"
2,3,Mie bakso,114,5,3,16,"12,000"
3,4,Mie celon,102,3,2,18,"11,000"
4,5,Mie pangsit basah,105,6,5,9,"13,000"
...,...,...,...,...,...,...,...
113,119,Pempek kulit,243,5,7,39,"18,000"
114,120,Pempek telur,156,5,3,28,"22,000"
115,121,Pempek tenggiri,173,7,1,33,"28,000"
116,122,"Telur ayam, dadar, masakan",251,16,19,1,"3,000"


In [34]:
snack_data

,No,Food Name,Calorie,Carbohidrat,Fat,Protein,Price
0,1,Dodol bali,298,"3,7","2,1","65,9",20000
1,2,Dodol galamai,348,"0,4","3,8","76,1",25000
2,3,Dodol kedondong,331,"1,9","7,6","63,8",15000
3,4,Dodol manado,373,"4,3","16,5","51,9",18000
4,5,Emping beras,403,"10,6","2,2","85,3",12000
...,...,...,...,...,...,...,...
160,161,"Mentimun Suri, segar",16,"1,3","0,0","2,1",4000
161,162,"Susu kambing, segar",64,"4,3","2,3","6,6",15000
162,163,"Susu kerbau, segar",160,"6,3","12,0","7,1",18000
163,164,"Susu kuda, segar",40,"2,0","1,1","5,4",16000


In [35]:
dataset_pengguna.loc[dataset_pengguna['Jenis Kelamin'] == 'L', 'Faktor Aktivitas'] = dataset_pengguna.loc[dataset_pengguna['Jenis Kelamin'] == 'L', 'Faktor Aktivitas'].replace({'Istirahat di tempat tidur': 1.2, 'Kerja sangat ringan': 1.4, 'Kerja ringan': 1.5, 'Kerja ringan-sedang': 1.7, 'Kerja sedang': 1.8, 'Kerja berat': 2.1, 'Kerja berat sekali': 2.3})
dataset_pengguna.loc[dataset_pengguna['Jenis Kelamin'] == 'P', 'Faktor Aktivitas'] = dataset_pengguna.loc[dataset_pengguna['Jenis Kelamin'] == 'P', 'Faktor Aktivitas'].replace({'Istirahat di tempat tidur': 1.2, 'Kerja sangat ringan': 1.4, 'Kerja ringan': 1.5, 'Kerja ringan-sedang': 1.6, 'Kerja sedang': 1.7, 'Kerja berat': 1.8, 'Kerja berat sekali': 2.0})
dataset_pengguna['Faktor Stress'] = dataset_pengguna['Faktor Stress'].replace({'Stress Level': 1.1, 'Tidak Ada stres': 1.1, 'Stres Ringan': 1.2, 'Stres Sedang': 1.4,'Stres Berat': 1.5,  'Stres berat': 1.5, 'Stres Sangat Berat': 1.7, 'Stres sangat berat': 1.7})

In [36]:
dataset_pengguna

,Kode User,Usia,BB,TB,Jenis Kelamin,Riwayat Penyakit,Riwayat Alergi,Faktor Aktivitas,Faktor Stress,disliked_foods,food_allergen
0,1,10,25,110,L,Maag,Kacang,1.2,1.1,Pempek,udang
1,2,20,55,150,P,Diabetes,Udang,1.5,1.5,Ayam Goreng,udang
2,3,30,58,155,L,Maag,Seafood,2.1,1.4,Martabak Manis,kerang
3,4,40,60,160,P,Diabetes,-,1.5,1.4,Sate Padang,kacang
4,5,50,49,148,L,-,-,2.1,1.5,Martabak Manis,kacang
...,...,...,...,...,...,...,...,...,...,...,...
200,201,41,83,180,L,-,-,1.8,1.7,-,-
201,202,46,85,182,P,maag,Susu Sapi,1.8,1.4,-,-
202,203,48,83,178,L,-,Udang,2.3,1.2,-,-
203,204,42,73,175,P,maag,-,1.4,1.2,-,-


In [37]:
dataset_pengguna['CalorieNeeded'] = dataset_pengguna.apply(lambda row: (66 + (13.7 * row['BB']) + (5 * row['TB'])
 - (6.8 * row['Usia'])) * row['Faktor Aktivitas'] * row['Faktor Stress'] if row['Jenis Kelamin'] == 'L'
 else (655 + (9.6 * row['BB']) + (1.8 * row['TB']) - (4.7 * row['Usia'])) * row['Faktor Aktivitas'] * row['Faktor Stress'], axis=1)

In [38]:
# Pisahkan atribut pengguna
dataset_user = dataset_pengguna.iloc[:, [0, 5, 6, 9, 10, 11]]  # Kode user, Riwayat Penyakit, Riwayat Alergi, CalorieNeeded

# **Input Data User**
Input user data from dataset user

In [39]:
import tensorflow.compat.v1 as tf
# User's calorie needs
# note: use tb, bb, fs (cek di collab capstone)
#user_calorie_needs = 5000

# Convert food database to numpy arrays
food_ids = np.array(list(food_database.keys()))
food_names = np.array([food_database[id]['Food Name']for id in food_ids])
calorie_values = np.array([food_database[id]['Calorie'] for id in food_ids])
disliked_foods = dataset_user['disliked_foods']
user_allergies = dataset_user['food_allergen']
food_carbohydrates = np.array([food_database[id]['Carbohidrat'] for id in food_ids])
food_protein = np.array([food_database[id]['Protein'] for id in food_ids])
food_fat = np.array([food_database[id]['Fat'] for id in food_ids])

# Convert food database to numpy arrays
snack_ids = np.array(list(snack_database.keys()))
snack_names = np.array([snack_database[id]['Food Name']for id in food_ids])
calorie_values2 = np.array([snack_database[id]['Calorie'] for id in food_ids])
snack_carbohydrates = np.array([snack_database[id]['Carbohidrat'] for id in food_ids])
snack_protein = np.array([snack_database[id]['Protein'] for id in food_ids])
snack_fat = np.array([snack_database[id]['Fat'] for id in food_ids])

In [40]:
kode_user = input()
user_calorie_needs = dataset_user[dataset_user['Kode User'] == int(kode_user)]
user_calorie_needs = dataset_user['CalorieNeeded'].values[0]

1


# **Recomendation Food**
Get Recomendation for Breakfast, Lunch, Dinner, Snack pagi and Snack Siang as technically

note: this is just for collect dataset

In [46]:
# Foods to be excluded
# use database
# pr: connected to sql

# Extract the numerical value from the array
user_calorie_needs = float(user_calorie_needs)
def generate_recommendations():
  # TensorFlow graph
  graph = tf.Graph()

  with graph.as_default():
    # Placeholder for user's calorie needs
      calorie_needs_placeholder = tf.placeholder(tf.float32)

    # Calculate absolute differences between calorie values and user's needs
      differences = tf.abs(calorie_values - calorie_needs_placeholder)

    # Sort the indices based on differences
      sorted_indices = tf.argsort(differences)

    # Gather the recommended food names
      recommendations = tf.gather(food_names, sorted_indices)
      recommendations_snack = tf.gather(snack_names, sorted_indices)

    # TensorFlow session
      with tf.Session() as sess:
        # Generate food recommendations
        # Breakfast: 25% user calori
        # lunch: 30% user calori
        # dinner: 25% user calori
          recommended_foods = sess.run(recommendations, feed_dict={calorie_needs_placeholder: user_calorie_needs})
          recommended_snack = sess.run(recommendations_snack, feed_dict={calorie_needs_placeholder: user_calorie_needs})

          np.random.shuffle(recommended_foods)
          np.random.shuffle(recommended_snack)

          filtered_recommendations = []
          #for food
          for food in recommended_foods:
            food_id = food_ids[np.where(food_names == food.decode('utf-8'))]
            food_id = food_id[0]
            food_calories = food_database[food_id]['Calorie']

            # Check if the food contains any allergens

            if food not in disliked_foods:
              if food_calories <= 0.25 * user_calorie_needs and len(filtered_recommendations) < 1:
                    filtered_recommendations.append(food)
              elif food_calories <= 0.30 * user_calorie_needs and len(filtered_recommendations) < 2:
                    filtered_recommendations.append(food)
              elif food_calories <= 0.25 * user_calorie_needs and len(filtered_recommendations) < 3:
                    filtered_recommendations.append(food)

          #for snack
          for snack in recommended_snack:
            snack_id = snack_ids[np.where(snack_names == snack.decode('utf-8'))]
            snack_id = snack_id[0]
            snack_calories = snack_database[snack_id]['Calorie']

            # Check if the food contains any allergens

            if snack not in disliked_foods:
              if snack_calories <= 0.10 * user_calorie_needs and len(filtered_recommendations) < 4:
                    filtered_recommendations.append(snack)
              elif snack_calories <= 0.10 * user_calorie_needs and len(filtered_recommendations) < 5:
                    filtered_recommendations.append(snack)

        # Display the filtered recommendations
          print("Filtered Recommendations:")
          print(f"Breakfast is: {filtered_recommendations[0].decode('utf-8')}")
          print(f"Lunch is: {filtered_recommendations[1].decode('utf-8')}")
          print(f"Dinner is: {filtered_recommendations[2].decode('utf-8')}")
          print(f"Snack Pagi is: {filtered_recommendations[3].decode('utf-8')}")
          print(f"Snack Siang is: {filtered_recommendations[4].decode('utf-8')}")


# Generate initial recommendations
generate_recommendations()

Filtered Recommendations:
Breakfast is: Tahu telur
Lunch is: Nasi gemuk
Dinner is: Soto bandung, masakan
Snack Pagi is: Mangga gedung, segar
Snack Siang is: Kacang panjang, kukus


In [47]:
generate_recommendations()

Filtered Recommendations:
Breakfast is: Soto madura, masakan
Lunch is: Pempek kelesan
Dinner is: Nasi gemuk
Snack Pagi is: Kangkung, kukus
Snack Siang is: Daun ubi merah, kukus
